# Pre-Class Assignment 5

<div style="text-align: right"> Emily Elizondo </div>
<div style="text-align: right"> Due 1/25/23 at 11:59pm </div>

### Coding Assignment

Implement the multivariable Secant method (not Newton’s method - calculate the derivative numerically, modifying code you’ve written in the past!) to find the joint root of f1 and f2 (which can be found by inspection to be at x = y = 0). Choose a starting point that is near zero but not precisely at zero (say, x = y = 1), and move progressively further away. How does that impact convergence and number of iterations? Compare this to the root method in the scipy.optimize package. How do your results compare, particularly as you experiment with different methods in root?


In [1]:
import numpy as np
import scipy.linalg
import scipy.optimize

In [2]:
'''
Two simple 2D functions; their joint root is (x0,x1) = (0,0).

Takes in a 2-element list or numpy array of (x0, x1) values, which should be floats.

Returns a 2-element list that is the value of both functions at that (x0,x1) position.
'''
def combined_functions(x):
    f1 = np.exp(-(x[1]**2+x[0]**2)/4.0)*x[0]**2
    f2 = 4.0*x[0]**2+1.5*x[1]**2
    return [f1,f2]

In [9]:
#this function numerically solves for the partial derivatives of two functions using the 2-point stencil
def numerical_diff(functions, x, h, n):
    
    x1 = x[0]
    x2 = x[1]
   
    #computing partial derivatives for each function with resect to each point using the 2-point stencil
    if n == 1:
        df1_dx1 = (functions((x1+h,x2))[0] - functions((x1,x2)))[0]/h
        df1_dx2 = (functions((x1,x2+h))[0] - functions((x1,x2)))[0]/h
        df2_dx1 = (functions((x1+h,x2))[1] - functions((x1,x2))[1])/h
        df2_dx2 = (functions((x1,x2+h))[1] - functions((x1,x2))[1])/h

    elif n == 2:
        df1_dx1 = (functions((x1+h,x2))[0] - functions((x1-h,x2))[0]) / (2*h)
        df1_dx2 = (functions((x1,x2+h))[0] - functions((x1,x2-h))[0]) / (2*h)
        df2_dx1 = (functions((x1+h,x2))[1] - functions((x1-h,x2))[1]) / (2*h)
        df2_dx2 = (functions((x1,x2+h))[1] - functions((x1,x2-h))[1]) / (2*h)

    #creating a matrix to place the partial derivatives in 
    A = np.zeros((2,2))
    
    #placing derivative in matrix 'A'
    A[0,0] = df1_dx1
    A[0,1] = df1_dx2
    A[1,0] = df2_dx1
    A[1,1] = df2_dx2
    
    return(A)

#this function finds the joint root of two privided functions using the secant method, given an initial guess
def secant_multivar(functions, guess, h, delta):
    
    iteration = 0      #initialization to keep track of the # of interations
    dx = [0.1,0.1]     #initializing change in x
    
    while abs(dx[0]) > delta or abs(dx[1]) > delta:
        
        A = numerical_diff(functions, guess, h, 2)        #copmuting matrix of partial derivatives
        
        
        dx = scipy.linalg.solve(A, functions(guess))   # solve the matrix equation A(dx) = b      
        guess = guess - dx                             # updating guess for root
        
        
        iteration += 1        #updating iteration
    
    root = guess
    
    return guess, iteration
        

In [10]:
#using initial guess (x0,x1) = (1,1)
secant_multivar(combined_functions, [1,1], 1e-6, 1e-5)

(array([1.67645177e-06, 8.06018773e-06]), 18)

In [11]:
#using initial guess (x0,x1) = (3,3)
secant_multivar(combined_functions, [3,3], 1e-6, 1e-5)

(array([1.47467685e-07, 5.27936043e-06]), 21)

In [12]:
#using initial guess (x0,x1) = (5,5)
#this is where it breaks
secant_multivar(combined_functions, [5,5], 1e-6, 1e-5)

/var/folders/ml/lry2ncps4nz1285ln669w9kr0000gn/T/ipykernel_85011/3920442355.py:42: LinAlgWarning: Ill-conditioned matrix (rcond=1.74543e-18): result may not be accurate.
  dx = scipy.linalg.solve(A, functions(guess))   # solve the matrix equation A(dx) = b
/var/folders/ml/lry2ncps4nz1285ln669w9kr0000gn/T/ipykernel_85011/3920442355.py:42: LinAlgWarning: Ill-conditioned matrix (rcond=1.44227e-32): result may not be accurate.
  dx = scipy.linalg.solve(A, functions(guess))   # solve the matrix equation A(dx) = b
/var/folders/ml/lry2ncps4nz1285ln669w9kr0000gn/T/ipykernel_85011/3920442355.py:42: LinAlgWarning: Ill-conditioned matrix (rcond=1.46433e-89): result may not be accurate.
  dx = scipy.linalg.solve(A, functions(guess))   # solve the matrix equation A(dx) = b


LinAlgError: Matrix is singular.

#### Using root fucntion in scipy optemize package

In [57]:
#using initial guess (x0,x1) = (1,1)
scipy.optimize.root(combined_functions, [1,1], method = 'hybr')

    fjac: array([[ 0.1477199 , -0.98902924],
       [ 0.98902924,  0.1477199 ]])
     fun: array([9.53824561e-233, 7.09956133e-232])
 message: 'The number of calls to function has reached maxfev = 600.'
    nfev: 600
     qtf: array([-1.80141024e-231,  5.21540357e-232])
       r: array([-1.13500212e-109,  7.49132647e-110, -5.70298487e-116])
  status: 2
 success: False
       x: array([-9.76639422e-117, -1.47969886e-116])

In [60]:
#using initial guess (x0,x1) = (1,1)
scipy.optimize.root(combined_functions, [1,1], method = 'broyden2')

     fun: array([3.06882815e-06, 3.91560708e+01])
 message: 'The maximum number of iterations allowed has been reached.'
     nit: 300
  status: 2
 success: False
       x: array([-0.04575119,  5.10866572])

In [64]:
#using initial guess (x0,x1) = (1,1)
scipy.optimize.root(combined_functions, [1,1], method = 'krylov')

     fun: array([4.60461799e-08, 1.78082567e-06])
 message: 'A solution was found at the specified tolerance.'
     nit: 12
  status: 1
 success: True
       x: array([0.00021458, 0.00103171])